#### Create a temporary directory to hold a local package.

In [ ]:
%goflags --cover --covermode=set

In [ ]:
!*TMP_PKG=$(mktemp -d /tmp/gonb_tests_gowork_XXXXXXXX) \
  && echo -n "${TMP_PKG}" > TMP_PKG.txt
!*cat TMP_PKG.txt

#### Create dummy Go module

In [ ]:
func Assert(err error) {
    if err != nil {
        panic(err)
    }
}

func Must[T any](v T, err error) T {
    Assert(err)
    return v
}

var TmpPkg string

func init_TmpPkg() {
    fPath := path.Join(os.Getenv("GONB_TMP_DIR"), "TMP_PKG.txt")
    TmpPkg = string(Must(os.ReadFile(fPath)))
}

%%
fmt.Printf("Temporary test package: %s\n", TmpPkg)

Assert(os.WriteFile(path.Join(TmpPkg, "go.mod"), []byte("module a.com/a/pkg\n\ngo 1.21\n"),0660))
Assert(os.WriteFile(path.Join(TmpPkg, "pkg.go"), []byte(
    "package pkg\n\nfunc A() string { return \"A\" }\n"),0660))

In [ ]:
!*cat "$(cat TMP_PKG.txt)"/*

#### Test creating `go.work` and using `%goworkfix`

In [ ]:
!*rm -f go.work && go work init && go work use . "$(cat TMP_PKG.txt)"
%goworkfix

In [ ]:
!*cat go.mod | sed 's|'"$(cat TMP_PKG.txt)"'|TMP_PKG|g;'

#### Check for tracked directories

In [ ]:
%track

In [ ]:
!*rm -f go.work
!*go mod edit -dropreplace=a.com/a/pkg
!*cat go.mod

In [ ]:
%untrack /tmp/gonb_tests_gowork_...
%track

#### Clean up temporary directory

In [ ]:
!*rm -r "$(cat TMP_PKG.txt)"